In [4]:
!pip install torch torchvision transformers pillow

In [3]:
import kagglehub
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from PIL import Image

dataset_path = kagglehub.dataset_download("yasinpratomo/wound-dataset")
print("Path to dataset files:", dataset_path)


train_data_path = os.path.join(dataset_path, "Wound_dataset")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 8
num_epochs = 5

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(train_data_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
class_names = train_dataset.classes
print("Classes detected:", class_names)
num_classes = len(class_names)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

def predict_wound(image_path):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, pred = torch.max(outputs, 1)
        label = class_names[pred.item()]

    description_map = {
        "Abrasions": "Minor skin damage. Clean and cover the area.",
        "Bruises": "Skin burn detected. Seek medical attention if severe.",
        "Burns": "Skin burn detected. Seek medical attention if severe.",
        "Cut": "Cut detected. May require dressing or sutures.",
        "Ingrown_nails": "Minor skin damage. Clean and cover the area.",
        "Laceration": "Cut detected. May require dressing or sutures.",
        "Stab_wound": "Cut detected. May require dressing or sutures."
    }
    return label, description_map.get(label, "Wound detected.")

test_image_path = os.path.join(train_data_path, "Abrasions", os.listdir(os.path.join(train_data_path, "Abrasions"))[0])
label, desc = predict_wound(test_image_path)
print("Predicted Wound Type:", label)
print("Description:", desc)

Using Colab cache for faster access to the 'wound-dataset' dataset.
Path to dataset files: /kaggle/input/wound-dataset
Classes detected: ['Abrasions', 'Bruises', 'Burns', 'Cut', 'Ingrown_nails', 'Laceration', 'Stab_wound']


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/5], Loss: 1.9264
Epoch [2/5], Loss: 1.5428
Epoch [3/5], Loss: 1.3576
Epoch [4/5], Loss: 1.2208
Epoch [5/5], Loss: 1.1742
Predicted Wound Type: Bruises
Description: Skin burn detected. Seek medical attention if severe.


In [5]:
import os
from PIL import Image
import torch
from torchvision import transforms
import kagglehub


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


def predict_wound(model, image_path, class_names):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(image)
        _, pred = torch.max(outputs, 1)
        label = class_names[pred.item()]


    description_map = {
        "Abrasions": "Minor skin damage. Clean and cover the area.",
        "Bruises": "Skin burn detected. Seek medical attention if severe.",
        "Burns": "Skin burn detected. Seek medical attention if severe.",
        "Cut": "Cut detected. May require dressing or sutures.",
        "Ingrown_nails": "Minor skin damage. Clean and cover the area.",
        "Laceration": "Cut detected. May require dressing or sutures.",
        "Stab_wound": "Cut detected. May require dressing or sutures."
    }
    return label, description_map.get(label, "Wound detected.")

dataset_path = kagglehub.dataset_download("yasinpratomo/wound-dataset")
data_path = os.path.join(dataset_path, "Wound_dataset")


class_names = os.listdir(data_path)



for cls in class_names:
    cls_path = os.path.join(data_path, cls)

    if os.path.isdir(cls_path):
        for img_name in os.listdir(cls_path):
            img_path = os.path.join(cls_path, img_name)

            if os.path.isfile(img_path):
                label, desc = predict_wound(model, img_path, class_names)
                print(f"Image: {img_name}")
                print(f"Predicted Wound Type: {label}")
                print(f"Description: {desc}")
                print("-" * 50)

Using Colab cache for faster access to the 'wound-dataset' dataset.
Image: abrasions (85).jpg
Predicted Wound Type: Stab_wound
Description: Cut detected. May require dressing or sutures.
--------------------------------------------------
Image: abrasions (55).jpg
Predicted Wound Type: Burns
Description: Skin burn detected. Seek medical attention if severe.
--------------------------------------------------
Image: abrasions (28).jpg
Predicted Wound Type: Laceration
Description: Cut detected. May require dressing or sutures.
--------------------------------------------------
Image: abrasions (9).jpg
Predicted Wound Type: Laceration
Description: Cut detected. May require dressing or sutures.
--------------------------------------------------
Image: abrasions (69).jpg
Predicted Wound Type: Stab_wound
Description: Cut detected. May require dressing or sutures.
--------------------------------------------------
Image: abrasions (39).jpg
Predicted Wound Type: Stab_wound
Description: Cut detec